In [1]:
import dash
from dash import html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import psycopg2

# Connessione a PostgreSQL
conn = psycopg2.connect(database="postgres", user="postgres", password="932197", host="127.0.0.1", port="5432")

app = dash.Dash(__name__)

def query_db():
    df = pd.read_sql_query("SELECT * FROM Prova LIMIT 10;", conn)
    return df

# Layout iniziale
app.layout = html.Div([
    dcc.Dropdown(
        id='dropdown',
        options=[
            {'label': 'Opzione 1', 'value': 'val1'},
            {'label': 'Opzione 2', 'value': 'val2'}
        ],
        value='val1'
    ),
    dcc.Graph(id='graph')
])

# Callback per aggiornare il grafico
@app.callback(
    Output('graph', 'figure'),
    Input('dropdown', 'value')
)
def update_graph(selected_value):
    df = query_db()
    # Filtra o trasforma il DataFrame in base a selected_value
    fig = px.bar(df, x='campo1', y='campo2')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


ModuleNotFoundError: No module named 'dash'